In [1]:
import networkx as nx
import json
import numpy as np


def Para_Esti(N, mu):
    if N!=986 and N!=23748:
        print(N, mu)
    else:
        print(N)
    
    with open(f'graph_{N}_{mu}.edgelist', 'r') as file:
            lines = file.readlines()

    ### Process the lines and create a list of number pairs
    edge_list = []
    for line in lines:
        pair = tuple(map(int, line.strip().split()))
        edge_list.append(pair)

    ### 新建一个图 
    G = nx.Graph()
    ### 向图添加点和边
    sorted_nodes = sorted(set(range(N)))
    G.add_nodes_from(sorted_nodes)
    G.add_edges_from(edge_list)

    #betweenness = list(nx.betweenness_centrality(G).values())
    #betweenness_output_file = f"graph_{N}_{mu}.between"

    components = list(nx.connected_components(G))

    # Find the largest component
    largest_component = max(components, key=len)

    from networkx.algorithms.community.quality import modularity


    global_clustering_coefficient = nx.average_clustering(G)

    print("Global Clustering Coefficient:", global_clustering_coefficient)

    # Read group assignments from a text file
    with open(f'graph_{N}_0.01.membership', 'r') as file:
        group_assignments = [int(line.strip()) for line in file]

    # Create communities based on group assignments
    communities = {}
    for node, group_id in enumerate(group_assignments):
        if group_id in communities:
            communities[group_id].add(node)
        else:
            communities[group_id] = {node}

    # Calculate modularity
    modularity_value = modularity(G, list(communities.values()))
    print("Modularity:", modularity_value)

    # Calculate conductance for each community
    conductance_values = []

    for group_id, community in communities.items():
        external_edges = len(set(edge for edge in G.edges(community) if not (edge[0] in community and edge[1] in community)))
        internal_edges = len(G.edges(community))
        conductance_value = external_edges / (external_edges + internal_edges)
        conductance_values.append(conductance_value)
        #print(f"Conductance for Group {group_id}:", conductance_value)

    mean_conductance = sum(conductance_values) / len(conductance_values)
    print("Mean Conductance:", mean_conductance)

In [2]:
Para_Esti(986, 0.01)

986
Global Clustering Coefficient: 0.40705044751953845
Modularity: 0.31304012017785127
Mean Conductance: 0.455387417940474


In [3]:
Para_Esti(1000, 0.01)

1000 0.01
Global Clustering Coefficient: 0.5799343368273093
Modularity: 0.9049546963208295
Mean Conductance: 0.012298757379969438


In [4]:
Para_Esti(10000, 0.01)

10000 0.01
Global Clustering Coefficient: 0.4106393337522117
Modularity: 0.9176956151414903
Mean Conductance: 0.01673180033950308


In [5]:
Para_Esti(23748, 0.01)

23748
Global Clustering Coefficient: 0.36040401927254917
Modularity: 0.1708071608921425
Mean Conductance: 0.45370298208977905
